In [1]:
import pandas as pd
import numpy as np
import os
import glob
from tasks import compute_index
from tasks.post_process import compare_2020_2019_data_report

# For the results

In [2]:
def make_2019_2020_correlation_report():
    
    data_2019 = pd.read_csv('data/2019_archive/result.csv').assign(version='v_2019')
    data_2020 = pd.read_csv('data/full_data/result.csv').assign(version='v_2020')
    data = pd.concat([data_2019, data_2020], axis=0).dropna(subset=['Value'])
    ISO_with_index = data.query("Aggregation == 'Index'").dropna().ISO.unique() # Select only ISOs where the full index is computed to remove some noise
    data = data[data.ISO.isin(ISO_with_index)]
    
    pivoted_data = data.pivot(index=['Variable', 'Aggregation', 'ISO', 'Year'], columns=['version'], values='Value')
    
    corr_by_var = pivoted_data.groupby(['Variable', 'Aggregation']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')
    corr_by_var_ISO = pivoted_data.groupby(['Variable', 'Aggregation', 'ISO']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')


    return pivoted_data, corr_by_var.reset_index(), corr_by_var_ISO.reset_index()


In [3]:
data, corr_by_var , corr_by_var_ISO = make_2019_2020_correlation_report()

In [4]:
corr_by_var.query('Aggregation == "Indicator" and corr < 0.9')

,Variable,Aggregation,corr
47,GE1,Indicator,0.798867
51,GE3,Indicator,0.898253
58,GN1,Indicator,0.209781
76,SE2,Indicator,0.438966


In [7]:
corr_by_var.query('Aggregation == "Indicator_normed" and corr < 0.9')

,Variable,Aggregation,corr
48,GE1,Indicator_normed,0.770216
59,GN1,Indicator_normed,0.584670
83,SL1,Indicator_normed,0.695073


In [8]:
corr_by_var.query('Aggregation == "Category" and corr < 0.9')

,Variable,Aggregation,corr
57,GN,Category,0.584670
81,SL,Category,0.762107


In [15]:
data.loc['Index', :, :, 2019].sort_values(by="v_2020").tail(30)

version                          v_2019     v_2020
Variable Aggregation ISO Year                     
Index    Index       FRA 2019.0   68.84  66.538515
                     POL 2019.0   68.89  66.943874
                     NLD 2019.0   67.06  67.821080
                     HRV 2019.0   67.83  67.839135
                     SVN 2019.0   71.00  67.912221
                     PRT 2019.0   70.36  67.963484
                     ROU 2019.0   68.39  68.060782
                     CHE 2019.0   73.20  68.358815
                     LVA 2019.0   69.11  68.378075
                     HUN 2019.0   71.39  68.728887
                     ITA 2019.0   68.05  69.753053
                     LTU 2019.0   71.58  70.544977
                     EST 2019.0   69.62  72.192818
                     DEU 2019.0   75.82  72.837620
                     SVK 2019.0   74.23  73.047368
                     AUT 2019.0   75.21  73.418919
                     FIN 2019.0   74.48  74.401023
                     CZE 2019.0   76.73  74.673817
                     DNK 2019.0   76.76  75.918352
                     SWE 2019.0   78.71  78.312649
                     AFG 2019.0   27.62        NaN
                     BIH 2019.0   39.31        NaN
                     CPV 2019.0   47.17        NaN
                     DZA 2019.0   28.01        NaN
                     IRN 2019.0   48.44        NaN
                     MLT 2019.0   31.76        NaN
                     MNE 2019.0   43.78        NaN
                     TJK 2019.0   29.03        NaN
                     TTO 2019.0   30.29        NaN
                     ZMB 2019.0   31.88        NaN

In [14]:
data.loc['Index', :, :, 2019].sort_values(by="v_2019").tail(30)

version                          v_2019     v_2020
Variable Aggregation ISO Year                     
Index    Index       GRC 2019.0   61.84  61.362557
                     ESP 2019.0   66.75  66.002642
                     GBR 2019.0   66.86  66.395992
                     NLD 2019.0   67.06  67.821080
                     HRV 2019.0   67.83  67.839135
                     NOR 2019.0   68.00  63.499721
                     ITA 2019.0   68.05  69.753053
                     ROU 2019.0   68.39  68.060782
                     FRA 2019.0   68.84  66.538515
                     POL 2019.0   68.89  66.943874
                     BEL 2019.0   69.07  66.079557
                     LVA 2019.0   69.11  68.378075
                     EST 2019.0   69.62  72.192818
                     PRT 2019.0   70.36  67.963484
                     SVN 2019.0   71.00  67.912221
                     HUN 2019.0   71.39  68.728887
                     LTU 2019.0   71.58  70.544977
                     CHE 2019.0   73.20  68.358815
                     SVK 2019.0   74.23  73.047368
                     FIN 2019.0   74.48  74.401023
                     AUT 2019.0   75.21  73.418919
                     DEU 2019.0   75.82  72.837620
                     CZE 2019.0   76.73  74.673817
                     DNK 2019.0   76.76  75.918352
                     SWE 2019.0   78.71  78.312649
                     FJI 2019.0     NaN  55.485660
                     GMB 2019.0     NaN  30.433462
                     OMN 2019.0     NaN  35.689314
                     RWA 2019.0     NaN  31.798439
                     SWZ 2019.0     NaN  33.066370

In [10]:
data.loc[:, 'Category', 'AFG', 2020]

version,v_2019,v_2020
Variable,,
AB,43.83,36.555367
BE,9.21,8.575380
CV,37.81,36.818737
EE,70.57,68.108864
EQ,71.19,71.996948
EW,NaN,21.173672
GB,40.11,40.109690
GE,79.24,78.489210
GJ,1.00,1.000000


In [ ]:
test = pd.read_csv('data/full_data/data.csv')

In [13]:
test.query("Indicator == 'ME1' and ISO == 'FRA'")

NameError: name 'test' is not defined

In [ ]:
pd.read_csv('data/indicator/SL1/processed/SL1_origin.M.csv').query("ISO == 'NLD'")

In [ ]:
data.loc['GN1', :, 'FRA', ]

In [ ]:
data.loc['ME1', :, 'USA', ]

In [ ]:
data.loc['SL1', :, 'USA', 2020]

In [ ]:
data.loc[:, 'Dimension', 'USA', 2020]

# SL1:

Problem from last year's version corrected today (Large outliers that where not accounted for and that are now removed.)


New data includes less points some of the values are not computed (eg the outliers, HKG, SGP etc)

**Conclusion**: Using SL1 v_2020 seems more appropriate 

In [ ]:
data.loc['SL1', 'Indicator_normed', : ,2019]['v_2019'].sort_values().head(15)

# GE1

Values are different from last year despite multiple APIs check. It is unclear why the values are different.

TO CHECK but not clear what's going on

In [ ]:
corr_by_var_ISO.query('Variable == "GE1" and Aggregation == "Indicator"').sort_values(by='corr').head(10)

In [ ]:
pd.read_csv('data/indicator/GE1/preprocessed/GE1.0_CW.csv')#.query('ISO == "FRA"')

In [ ]:
pd.read_csv('data/indicator/GE1/computed/GE1.csv').query('ISO == "FRA"')

In [ ]:
data.loc['GE1', 'Indicator', 'FRA']

In [ ]:
data.loc['GE1', 'Indicator', 'ITA' , :]#['v_2019'].sort_values().head(15)

In [ ]:
config = {
            "GGI_code": "GE1.0",
            "params": {
              "gas_ids[]": 326,
              "sector_ids[]": [1299],
              "source_ids[]": [111]
            }
        }

In [ ]:
from download.downloaders import download, CW_Downloader
from processing.api_preprocessors import CW_Preprocessor

In [ ]:
test = download(API_name="CW", path=None, config=config)
test = CW_Preprocessor('GE1.0').json_to_pandas(test['data'])

In [ ]:
pop = pd.read_csv('data/indicator/GE1/processed/GE1.1_WB.csv')

In [ ]:
(test.query('iso_code3== "FRA"').set_index('year').value * 1e6 / pop.query('ISO == "FRA"').set_index('Year')['Value']).dropna()

In [ ]:
data.loc['GE1', 'Indicator', 'FRA']

In [ ]:
pd.read_csv('data/indicator/GE1/processed/GE1.0_CW.csv')

In [ ]:
def process_GE1_0(df):
    df = df.copy()
    piv = df.pivot(index=['iso_code3', 'year'], columns=['sector'], values='value')
    piv['value'] = (piv['Total excluding LUCF'] - piv['Agriculture']) * 1e6
    piv = (
        piv['value'].reset_index().assign(Description='Total excluding LUCF and Agriculture')
                    .assign(data_source='CAIT _and_ PIK')
    )
    
    
    return piv.query('value > 0')

In [ ]:
process_GE1_0(test)

# GE3 

Same as GE1, not clear what the problem is and where it comes from


In [ ]:
config = {
            "GGI_code": "GE3.1",
            "params": {
                "gas_ids[]": [
                    327,
                    328,
                    329
                ],
                "sector_ids[]":[
                    1304,
                    1302
                ],
                "source_ids[]": 111
            }
        }


test = download(API_name="CW", path=None, config=config)


In [ ]:
test = CW_Preprocessor('GE2.0').json_to_pandas(test['data'])

In [ ]:
test

In [ ]:
CW_Downloader('https://www.climatewatchdata.org/api/v1/data/historical_emissions').get_CW_API_ids("sectors")

#  ME1 

Check for a different source

In [ ]:
data.loc['ME1', 'Indicator', 'RWA']

# GN1 

Values are one the right order of magnitude but change from one version to the other, should double check properly and redownload.

In [ ]:

def make_2019_2020_correlation_report_bis():
    
    data_2019 = pd.read_csv('data/2019_archive/data.csv').assign(version='v_2019')
    data_2020 = pd.read_csv('data/full_data/data.csv').assign(version='v_2020')
    data = pd.concat([data_2019, data_2020], axis=0).dropna(subset=['Value'])
    
    pivoted_data = data.pivot(index=['Indicator', 'ISO', 'Year'], columns=['version'], values='Value')
    
    corr_by_var = pivoted_data.groupby(['Indicator']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')
    corr_by_var_ISO = pivoted_data.groupby(['Indicator', 'ISO']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')


    return pivoted_data, corr_by_var.reset_index(), corr_by_var_ISO.reset_index()
data, corr_by_var , corr_by_var_ISO = make_2019_2020_correlation_report_bis()

In [ ]:
corr_by_var.query('corr < 0.9')

In [ ]:
corr_by_var_ISO.query("Indicator == 'GN1'").sort_values(by='corr').dropna().head(20)

In [ ]:
data.loc['GN1', 'ITA']

In [ ]:
corr_by_var_ISO.query('Variable == "GN1" and Aggregation == "Indicator"').sort_values(by='corr').dropna().tail(20)

# SE2

In [ ]:
corr_by_var_ISO.query('Variable == "SE2" and Aggregation == "Indicator"').sort_values(by='corr').head(30)

In [ ]:
data.loc['SE2', 'Indicator', 'RWA']